<a href="https://colab.research.google.com/github/lidar532/ppkgeotag/blob/2020-1020-dev/CORS_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/Gdrive')

# CORS_Lib.ipynb
( CORS_Lib_asof: 2021-0918-0746 )
Gather RINEX data sets from https://geodesy.noaa.gov/cors/rinex/



(c) 2020, 2021 C. W. Wright  


**Important Note:** High rate (1 Hz) is avaliable from CORS for only 30 days.  Data older than 30 days, at sampling intervals faster than every 30 seconds, can be had by contacting: https://www.avl.class.noaa.gov/saa/products/search?datatype_family=CORS

** Visit https://geodesy.noaa.gov/CORS/data.shtml for details on sp3 for gps & glonas, and also nav files.

In [ ]:
#@title Step 0. (Optional) Mount your Google Drive as /content/Gdrive { form-width: "35%", display-mode: "form" }
if __name__ == '__main__':
  from google.colab import drive
  !rm -rf /content/sample_data/
  drive.mount('/content/.gdrive')
  !ln -sf /content/.gdrive/My\ Drive/ /content/Gdrive
  print('Your Gdrive is mounted')

# Cells for Standalone usage.

In [ ]:
#@title Step 1. Run this cell first to load required defs and functions. { form-width: "35%", display-mode: "form" }

CORS_Lib_asof = 'CORS_Lib_asof: 2021-0918-0746'

#=================================================================================
#@title def teqc_install()  { form-width: "35%", display-mode: "form" }
# Get and install teqc
def teqc_install():
  import os
  import subprocess as sp
  state = 'initial.'
  os.chdir('/content/')
  print(f'teqc_install(): ', end='')
  #rv = sp.run(f'which teqc', shell=True)
  rv = !which teqc
  if rv == []:
    print('Downloading and installing Teqc from Unavco.  ', end='')
    sp.run(f'wget https://www.unavco.org/software/data-processing/teqc/development/teqc_Lx86_64s.zip', shell=True)
    sp.run(f'unzip teqc_Lx86_64s.zip', shell=True)
    sp.run(f'mv teqc /usr/local/bin', shell=True)
    sp.run(f'rm -rf teqc_Lx86_64s.zip', shell=True)
    state = ' Teqc installed.'
  else:
    a = 1;
    state='Teqc was already installed.'
  print(state)


#@title imports & Installs: Fetch CORS GNSS data for 1 or more sites on a date. { form-width: "35%", display-mode: "form" }
import datetime
import time
import os
import re
import glob
import pandas as pd
import multiprocessing
import ipywidgets as ipw
import subprocess as sp

print(CORS_Lib_asof)
teqc_install()

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

#=================================================================================
#@title def extract_year( date ) { form-width: "35%", display-mode: "form" }
def extract_year( date ):
  Year = date.split('/')[0]
  return Year


#=================================================================================
#@title def isTimeFormat(ts)  { form-width: "35%", display-mode: "form" }
def isTimeFormat(ts):
  """
  isTimeFormat(ts) checks the 'ts' input string for validity. A valid string is in
  this format: '13:34:56'.

  Returns True for a valid string,a nd False for invalid.
  
  """
  t = []
  rv = False;
  try:
      t = time.strptime(ts, '%H:%M:%S')
      rv = True;
  except ValueError:
      rv = False
  return rv

# testing code...
#if __name__ == '__main__':
#  for t in ['12:34:56',
#            '12:34',
#            '01:02:03',
#            '23:01:02',
#            '25:34:63']:
#    print(f'rv:{isTimeFormat(t)}')
#=================================================================================
#@title def compute_day_of_year(date_str) { form-width: "35%", display-mode: "form" }
def compute_day_of_year(date_str):
  Day = int(( datetime.datetime.strptime(date_str,'%Y/%m/%d') - datetime.datetime(int(date_str.split('/')[0]),1,1)).days + 1 )
  Day = f'{Day:03d}'
  return Day

#=================================================================================
#@title def CORS_get_all_station_data()  { form-width: "35%", display-mode: "form" }
def CORS_get_all_station_data(Project_Root_Folder, CORS_to_Fetch, date, UTC_Start_Time, UTC_End_Time):
  teqc_install()
  print(f'Fetching CORS Data for: {date}')
  Date = date.replace('-', '/')
  CORS_to_Fetch = CORS_to_Fetch.lower()
  CORS_to_Fetch = CORS_to_Fetch.replace(',', ' ')
  lst = CORS_to_Fetch.replace(',', ' ').split()
  get_CORS_SP3_Nav(Project_Root_Folder, Date )
  for sta in lst:
    rv = get_CORS_OBS_Data(Project_Root_Folder, Date, sta, UTC_Start_Time, UTC_End_Time )
    rv = CORS_get_station_coords(Project_Root_Folder+'/'+'CORS/'+sta+'/', sta)
    rv = CORS_get_station_datasheet(Project_Root_Folder+'/'+'CORS/'+sta+'/', sta)
  clean_up_CORS(Project_Root_Folder, Date)
  print('**************************************************************')
  print('*  All requested CORS data downloaded.  Operation completed. *')
  print('**************************************************************')




In [ ]:
#@title def get_CORS_OBS_Data( rootdir, date, Station, start, stop ) { form-width: "35%", display-mode: "form" }
#=================================================================================
def get_CORS_OBS_Data( rootdir, date, Station, start, stop ):
  """
  inputs: string rootdir,     Path where to store the CORS data.
          string date,        Date to fetch. ex: '2019/1/1'
          string Station,     CORS Station to fetch.
          string start,       Start time, ex '12:00:00'
          string stop         End time, ex. '15:30:01'
  returns
    Nothing.  Fetches data via ftp and places in local directory structure
  
  Example: 
    get_CORS_OBS_Data( '/content/2019-data', '2019/11/26', 'ncdu', '12:00:00', '15:30:01' )

  """
  Base_Date = date
  Day = compute_day_of_year(Base_Date)
  #Day = int(( datetime.datetime.strptime(Base_Date,'%Y/%m/%d') - datetime.datetime(int(Base_Date.split('/')[0]),1,1)).days + 1 )
  #Day = f'{Day:03d}'
  Base_FTP_Site = "https://geodesy.noaa.gov/corsdata/rinex/"
  try:
    os.makedirs(rootdir, exist_ok=True)
  except:
    print(f'{rootdir} already exists.')

  rootdir = rootdir+'/'
  Base_root = 'CORS'
  Year = extract_year(date)         # Year as 4 char ie 2021
  Year2 = Year[2:4]                 # year as 2 char ie 21
  YD = Year+'/'+Day
  print(f'\nGathering Data for: {Station}  for: {Base_Date}')
  # print(YD)
  print(sp.run(f'mkdir -p {rootdir}', shell=True))
  os.chdir(rootdir)
  print(sp.run(f'mkdir -p {Base_root}', shell=True))
  print(sp.run(f'mkdir -p {Base_root}/{Station}', shell=True))
  d = rootdir+Base_root+'/'+Station
  print(f'd={d}')
  os.chdir(d)
  url = f'{Base_FTP_Site}/{YD}/{Station}/{Station+str(Day)+"0"}.{Year2}o.gz'
  print(f'url: {url}')
  #os.chdir(str(rootdir+Base_root))
  #os.chdir(rootdir)
  os.chdir(d)
  rv = glob.glob('*.*o')
  ##print(f'Pre-existing RINEX observation files found: {rv}')
  if (rv) != []:                                    # When the obs file exits, don't overwrite it.
    print(f'OBS Files exist.  Skipped. rv:{rv}')
  else:                                             # Download the file.
    print(sp.run(f'wget -nv -nd -r --no-clobber {url}', shell=True))
    print(sp.run(f'gunzip -qr .', shell=True))
    obsfn = rootdir+Base_root+'/'+Station+'/'+Station+str(Day)+f'0.{Year2}o'
    print(f'Requested obs file: {obsfn}' )
    CORS_trim_to_time( obsfn, start, stop)
    print( f'Download of {Station} completed.')

    # Do the following block if the file was empty. This block will download
    # the hourly files, and concat them.  This will happen if the download is
    # on the same day as the data was collected, and the day may not be complete
    # yet.
    if os.path.getsize(obsfn) == 0:
      alt_ofn = datetime.datetime.now().strftime(f'{Station}%j-asof-%Y-%m%d-%H%M%S.%yo')
      print(f'**** WARNING ***** {obsfn} is an empty file.')   # Download hourly, and build up.
      print(f'Building: {alt_ofn}')
      url_for_parts = f'{Base_FTP_Site}/{YD}/{Station}/'
      print(f'url_for_parts:{url_for_parts}')
      print(sp.run(f'wget -r -np -nH --cut-dirs=6 {url_for_parts}', shell=True))
      print(sp.run(f'gunzip *', shell=True))
      print(sp.run(f'rm index.html* *.{Year2}S *.{Year2}d robots.txt* {obsfn}', shell=True))
      print(sp.run(f'teqc {Station}*[a-z].{Year2}o >{alt_ofn}', shell=True))
      CORS_trim_to_time( alt_ofn, start, stop)
      print(sp.run(f'rm {Station+str(Day)}[a-z].{Year2}o', shell=True))


#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
if __name__ == '__main__':
  get_CORS_OBS_Data_debug = False
  if get_CORS_OBS_Data_debug:
    ! rm -rf /content/test2; mkdir /content/test2
    get_CORS_OBS_Data( '/content/test2', '2021/09/18', 'ncdu', '12:00:00', '15:30:01' )


In [ ]:
#@title utctoweekseconds(utc,leapseconds) { form-width: "35%", display-mode: "form" }
#=================================================================================
def utctoweekseconds(utc,leapseconds):
    """ Returns the GPS week, the GPS day, and the seconds 
        and microseconds since the beginning of the GPS week """
    import datetime, calendar
    datetimeformat = "%Y-%m-%d %H:%M:%S"
    epoch = datetime.datetime.strptime("1980-01-06 00:00:00",datetimeformat)
    tdiff = utc -epoch  + datetime.timedelta(seconds=leapseconds)
    gpsweek = tdiff.days // 7 
    gpsdays = tdiff.days - 7*gpsweek         
    gpsseconds = tdiff.seconds + 86400* (tdiff.days -7*gpsweek) 
    return gpsweek,gpsdays,gpsseconds,tdiff.microseconds


In [ ]:
#@title def get_CORS_SP3_Nav( rootdir,  date ) { form-width: "35%", display-mode: "form" }
#=================================================================================
def get_CORS_SP3_Nav( rootdir,  date ):
  """
  get_CORS_SP3_Nav( rootdir,  date )

  inputs: string rootdir,     Path where to store the CORS data.
          string date         Date to fetch. ex: '2019/1/1'

  returns
    Nothing.  Fetches SP3 data via ftp and places in local directory structure

  get_CORS_SP3_Nav() gathers all of the sp3 files for a given date.  Sp3 files contain the
  GPS Satellite Ephemerides / Satellite & Station Clocks.  More information on these files
  can be found at: http://www.igs.org/products
  
  Example: 
    def get_CORS_SP3_Nav( '/content/2019-data', '2019/11/26' )


  """
  Base_Date = date
  Day = compute_day_of_year(Base_Date)
  #Day = int(( datetime.datetime.strptime(Base_Date,'%Y/%m/%d') - datetime.datetime(int(Base_Date.split('/')[0]),1,1)).days + 1 )
  #Day = f'{Day:03d}'
  Base_FTP_Site = "https://geodesy.noaa.gov/corsdata/rinex/"
  sp.run( f'mkdir -p {rootdir}', shell=True )
  rootdir = rootdir+'/'
  Base_root = 'CORS'
  Year = extract_year(date)
  YD = Year+'/'+Day
  Year2 = Year[2:4]
  print(YD)
  os.chdir(rootdir)
  rv = sp.run( f'mkdir -p {Base_root}', shell=True)
  print(rv)
  os.chdir(str(rootdir+Base_root))
  rv = sp.run(f'mkdir -p sp3 nav', shell=True)
  print(rv)

# {Base_FTP_Site}/{YD}/brdc{Day}/
# -np -nH 
  os.chdir('sp3')
  print(f'   Gathering Sp3 Data for: {Base_Date}')
  #cmd = f'wget -nv -nd  {Base_FTP_Site}/{YD}/brdc{Day}0.21g.gz {Base_FTP_Site}/{YD}/brdc{Day}0.21n.gz'
  #cmd = f'wget  --no-directories --cut-dirs=5 --accept ".gz"  {Base_FTP_Site}/{Year}/{Day}'
  #print(f'cmd:{cmd}')
  #sp.run( cmd, shell=True)

  os.chdir('../nav') 
  print(f'   Gathering Nav Data for: {Base_Date}')
  rv = sp.run(f'wget -nv -nd {Base_FTP_Site}/{YD}/', shell=True)
  print(rv)
  os.chdir(rootdir)
  rv = sp.run( f'gunzip -qrf .', shell=True)
  print(rv)

  import  re
  fn = '/content/test_sp3/CORS/nav/index.html'
  fn = f'{rootdir}/CORS/nav/index.html'
  files_to_get = []
  with open(fn) as f:
    for line in f.readlines():
      if '.gz' in line:
        ifn = re.split('<a href="(.*)">', line)[1]
        files_to_get.append( f' {Base_FTP_Site}/{YD}/'+ ifn)
        print(ifn)
  files_str = f' '.join(files_to_get)
  rv = sp.run(f'wget -nv -nd {files_str}', shell=True)
  print(rv)

  rv = sp.run(f'gunzip *; mv *{Year2}g *{Year2}n ./CORS/nav ;   mv *.sp3 ./CORS/sp3', shell=True)
  print(rv)

  print( f'Download of {date} Nav & SP3 completed.')



In [ ]:
#@title Test get_CORS_SP3_Nav() { form-width: "35%", display-mode: "form" }
#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
if __name__ == '__main__':
  get_CORS_SP3_Nav_debug = False
  if get_CORS_SP3_Nav_debug:
    test_path = '/content/test_xx3'
    ! rm -rf {test_path}; mkdir {test_path}
    get_CORS_SP3_Nav( f'{test_path}', '2021/09/18' )


In [ ]:
#=================================================================================
#@title def CORS_get_station_datasheet(dir, station) { form-width: "35%", display-mode: "form" }
# url: https://geodesy.noaa.gov/cgi-bin/ds_cors.prl?CorsSelected=|NCDU&CorsTypeSelected=Arp
def CORS_get_station_datasheet(dir, station):
  station = str.lower(station)
  os.makedirs(dir, exist_ok=True)
  rv = sp.run(f'curl  -B https://geodesy.noaa.gov/cgi-bin/ds_cors.prl?CorsSelected=|{str.upper(station)}&CorsTypeSelected=Arp > {dir}/{station}_datasheet.html', shell=True)
  return rv

#=================================================================================
#@title def CORS_get_station_coords(dir, station) { form-width: "35%", display-mode: "form" }
# url for data:
def CORS_get_station_coords(dir, station):
  station = str.lower(station)
  os.makedirs(dir, exist_ok=True)
  rv = sp.run(f'curl  -B https://geodesy.noaa.gov/corsdata/coord/coord_14/{station}_14.coord.txt > {dir}/{station}_14.coord.txt', shell=True)

#=================================================================================
#@title def clean_up_CORS(rdir, date) { form-width: "35%", display-mode: "form" }
def clean_up_CORS(rdir, date):
  """
  clean_up_CORS(rdir, date)
  Inputs:
  rdir      Directory to start the find command in.
  date      The date of the CORS files to cleanup.

  Outputs:
            Removes unnecessary files.
  Returns:
            None.

  Uses the Linux 'find' command to locate and remove unnecessary files from within a CORS station directory.  
  """
  Year = date.split('/')[0][2:4]
  sp.check_output(f'cd {rdir}; find -name *.md5 -delete; find -name *.{Year}d -delete; find -name *.md5* -delete; find -name *.{Year}S* -delete;', shell=True) 

#=================================================================================
#@title def CORS_trim_to_time(f, start, stop) { form-width: "35%", display-mode: "form" }
# Url for data: https://geodesy.noaa.gov/cors/coord/coord_14/
def CORS_trim_to_time(f, start, stop):
  """
  CORS_trim_to_time(f, start, stop)

  Inputs:
  f       RINEX File to trim.
  start   UTC Start time string. Ex '12:00:05'
  stop    UTC Stop  time string. Ex '12:00:05'

  Outputs:
  Overwrites the input file with the new trimmed version.

  Returns:
    None.

Trims a RINEX file (f) to be between the 'start' and 'stop' times given.
  """
  st = re.sub('[\:\-\.]', '', start)
  end = re.sub('[\:\-\.]', '', stop)
  print(f'CORS_trim_to_time(): f: {f}   start: {start}  stop: {stop}')
  sp.run(f'teqc +out tmp.txt -st {start} -e {stop} {f}', shell=True)
  sp.run(f'mv tmp.txt {f}', shell=True)



In [ ]:
#@title Step 3. (Optional) Preset CORS_Station_List. { form-width: "35%", display-mode: "form" }
if __name__ == '__main__':
  CORS_Station_List = "NCDU NCBI NCRT NCBX LOY2 LS03 NCEL NCCI NCSO NCBE NCJV NCCH NCFF NCSL" #@param ["MDAI hnpt ded2 demi dnrc dedo njcm loyw loyr loyf dene", "NCDU NCBI NCRT NCBX LOY2 LS03 NCEL NCCI NCSO NCBE NCJV NCCH NCFF NCSL", "ZEFR WACH BKVL BRTW"] {allow-input: true}
  print(f'Selected CORS Station(s): {CORS_Station_List}')


In [ ]:
#@title Step 3. (Optional) Preset CORS_Station_List for De. { form-width: "35%", display-mode: "form" }
if __name__ == '__main__':
  CORS_Station_List = "DEDO DED2 DNRC DED2 DEMI NJCM MDAI" #@param ["MDAI hnpt ded2 demi dnrc dedo njcm loyw loyr loyf dene", "NCDU NCBI NCRT NCBX LOY2 LS03 NCEL NCCI NCSO NCBE NCJV NCCH NCFF NCSL", "ZEFR WACH BKVL BRTW"] {allow-input: true}
  print(f'Selected CORS Station(s): {CORS_Station_List}')


In [ ]:
#@title  Step 4. (Jupyter Widgets) CORS_jw():   GUI to download CORS data.{ form-width: "35%", display-mode: "form" }
import numpy as np
import datetime


def CORS_jw():
  """
  CORS_jw() Uses Juypter Widget based GUI to download and save CORS RINEX data from NOAA. 

  CORS Site:  https://geodesy.noaa.gov/CORS/
  Juypter Widgets: https://ipywidgets.readthedocs.io/en/latest/index.html
  """
  global CORS_Station_List
  #@title def GET_sp3(b) {form-width: "25%"}
  # Reponse to SP3 button push
  def GET_sp3(b):
    print(f'GET_sp3 b={b}')
    global CORS_Date
    stations = sta_list.value
    CORS_Date =  date_w.value
    CORS_Date_str = CORS_Date.strftime("%Y/%m/%d")
    rv = get_CORS_SP3_Nav( proj_dir.value,  CORS_Date_str )

  #@title def GET_CORS(b)  {form-width: "25%"}
  # Respone to Download RINEX button
  def GET_CORS(b):
    global CORS_Date, CORS_Station_List
    stations = sta_list.value
    CORS_Date =  date_w.value
    CORS_Date_str = CORS_Date.strftime("%Y/%m/%d")
    Start_time_str = Start_time.value
    End_time_str = End_time.value
    with output:
      if isTimeFormat(Start_time_str) == False:
        print(f'{Start_time_str} is not a valid start time. The correct format is: "HH:MM:SS" ')
        return
      if isTimeFormat(End_time_str) == False:
        print(f'{End_time_str} is not a valid end time.  The correct format is: "HH:MM:SS"')
        return
      print(f'stations: {stations}')
      print(f'Project_dir:{proj_dir.value}')
      print(f'Date: {CORS_Date} {type(CORS_Date)}  {CORS_Date_str}')
      print(f'Start Time:{Start_time_str}')
      print(f'End Time:{End_time_str}')
      CORS_get_all_station_data(proj_dir.value, stations,  CORS_Date_str, Start_time_str, End_time_str)

  try:
    CORS_Date
  except:
    CORS_Date = datetime.datetime.now()

  try:
    Start_time_str
  except:
    Start_time_str = '00:00:00'

  try:
    End_time_str
  except:
    End_time_str = '23:59:59'

  try:
    CORS_Station_List
  except:
    CORS_Station_List = ''

  # Generate Jupyter Widgets entry widgets 
  proj_dir_lbl = ipw.Label( value="Project Path:")
  proj_dir = ipw.Text(value='/content/', layout=ipw.Layout(width='590px'))
  proj_dir_w = ipw.HBox( [proj_dir_lbl, proj_dir] )

  # same for Station list
  sta_lbl = ipw.Label(value="CORS Stations:")
  sta_list = ipw.Text(value=CORS_Station_List, disabled=False, layout=ipw.Layout(width='580px'))
  pd = ipw.HBox( [sta_lbl, sta_list  ])

  # generate a date picker widget and preload with CORS_Date if it exists
  date_w = ipw.DatePicker( description='Date:', disabled=False, value=CORS_Date, layout=ipw.Layout(width='250px') )

  # Generate Start and Stop time GUI entries
  Start_lbl  = ipw.Label( value="UTC Time Start:", layout=ipw.Layout(width='100px'))
  Start_time = ipw.Text(value=Start_time_str, layout=ipw.Layout(width='100px'))
  End_lbl    = ipw.Label( value="UTC End Time:", layout=ipw.Layout(width='100px'))
  End_time   = ipw.Text(value=End_time_str, layout=ipw.Layout(width='100px'))
  b = ipw.HBox([date_w, Start_lbl, Start_time, End_lbl, End_time])

  # Buttons
  sp3_b = ipw.Button(description = "Download SP3 Data", layout=ipw.Layout(width='180px') )
  sp3_b.style.button_color = 'lightgreen'

  go = ipw.Button(description = "Download CORS RINEX Data", layout=ipw.Layout(width='200px') )
  go.style.button_color = 'lightblue'
  buttons_w = ipw.Box([sp3_b, go])
  output = ipw.Output()
  go.on_click(GET_CORS)
  sp3_b.on_click(GET_sp3)
  display( ipw.HTML(value="<h1>Download CORS Data.</h1>"), pd, b, proj_dir_w, buttons_w, output)


if __name__ == '__main__':
  try:
    teqc_install()
    CORS_jw();
  except:
    print('You must Run the imports cell above first.')




# RINEX File Tools

In [ ]:
#@title Show Start, Stop times, stats of a RINEX obs file(s). { form-width: "35%", display-mode: "form" }
if __name__ == '__main__':
  Path_Names = "/content/Gdrive/Missions/Uas/2020-1006-MARKS/2020-1006-MARKS_GNSS/bases/MARKS_base/2020-1006-192851-Marks-Base-GP192832.obs" #@param {type:"string"}
  Path_Names = Path_Names.split()
  for fn in Path_Names:
    print(fn)
    junk = ! teqc +qc {fn} 2>/dev/null
    id  = junk.grep('4-character ID')
    beg = junk.grep('Time of ')[0]
    end = junk.grep('Time of ')[1]
    sumhead = junk.grep('first epoch')[0]
    summary = junk.grep('SUM')[0]
    print(f'{id}\n{beg}\n{end}\n{sumhead}\n{summary}\n')



In [ ]:
#@title Generate Quality Report(s) for a RINEX obs file(s)  (Teqc +qc). { form-width: "35%", display-mode: "form" }
if __name__ == '__main__':
  Path_Names = "/content/Gdrive/Missions/Uas/2020-1006-MARKS/2020-1006-MARKS_GNSS/bases/CORS/demi/demi2800.20o" #@param {type:"string"}
  Path_Names = Path_Names.split()
  for fn in Path_Names:
    print(f'\n\n{fn}')
    ! teqc +qc {fn} 2>/dev/null



# Developer Testing Tool Cells.

In [ ]:
#@title Testing: Remove Teqc binary. { form-width: "35%", display-mode: "form" }
if __name__ == '__main__':
  rv = !which teqc
  if rv != []:
    !rm -rf {rv[0]}
    print(f'{rv[0]} removed.')
  else:
    print(f'teqc was not installed.')



In [ ]:
#@title  Test CORS_get_all_station_data(Project_Root_Folder, CORS_to_Fetch, date, UTC_Start_Time, UTC_End_Time) { form-width: "35%", display-mode: "form" }
if __name__ == '__main__':
  CORS_to_Fetch = "ls03,    ncdu ncbi"
  CORS_get_all_station_data('/content/2020-0524', CORS_to_Fetch,  '2020/05/24', '12:00:00', '13:31:01')


In [ ]:
#@title Test (Colabs) CORS_get_station_datasheet() { form-width: "35%", display-mode: "form" }
if __name__ == '__main__':
  Project_Root_Folder = "/content/2020-0524" #@param {type:"string"}
  CORS_Station = "NCDU" #@param {type:"string"}
  CORS_Station = str.lower(CORS_Station)
  d = f'{Project_Root_Folder}/CORS/{CORS_Station}/'
  rv = CORS_get_station_datasheet(d, CORS_Station)

In [ ]:
#@title Test CORS_get_station_coords() { form-width: "35%", display-mode: "form" }
if __name__ == '__main__':
  Project_Root_Folder = "/content/Gdrive/Missions/2022-0613-Nardi-NJ-Supawna/2022-0613-Nardi-NJ-Supawna_GNSS/bases" #@param {type:"string"}
  CORS_Station = "NCDU" #@param {type:"string"}
  CORS_Station = str.lower(CORS_Station)
  d = f'{Project_Root_Folder}/CORS/{CORS_Station}/'
  CORS_get_station_coords('/content/2020-0524/CORS/ncdu', 'NCDU')

In [ ]:
#@title Test Google Colabs test code below getting all requested data for a site. { form-width: "35%", display-mode: "form" }


if __name__ == '__main__':
  teqc_install()
  Project_Root_Folder = "/content/2020-0524" #@param {type:"string"}
  #@markdown Enter a list of CORS Stations to download below. Separate with "," or space character.
  CORS_to_Fetch = "ls03,    ncdu ncbi ncbx ncci ncbe ncso" #@param {type:"string"}
  Date_to_Fetch = "2020-05-24" #@param {type:"date"}
  UTC_Start_Time = "16:32:20" #@param {type:"string"}
  UTC_End_Time   = "18:50:31" #@param {type:"string"}

  print(f'Fetching CORS Data for: {Date_to_Fetch}')
  Date = Date_to_Fetch.replace('-', '/')
  CORS_to_Fetch = CORS_to_Fetch.lower()
  CORS_to_Fetch = CORS_to_Fetch.replace(',', ' ')
  lst = CORS_to_Fetch.replace(',', ' ').split()
  get_CORS_SP3_Nav(Project_Root_Folder, Date )
  for sta in lst:
    rv = get_CORS_OBS_Data(Project_Root_Folder, Date, sta, UTC_Start_Time, UTC_End_Time )
    CORS_get_station_coords(Project_Root_Folder+'/'+'CORS/'+sta+'/', sta)
    CORS_get_station_datasheet(Project_Root_Folder+'/'+'CORS/'+sta+'/', sta)
  clean_up_CORS(Project_Root_Folder, Date)
  print('All requested CORS data downloaded.  Operation completed.')

#--------------------------------------------------------



In [ ]:
#@title Test Google Colabs test getting sp3 files. { form-width: "35%", display-mode: "form" }
if __name__ == '__main__':
  teqc_install()
  Project_Root_Folder = "/content/2020-0524" #@param {type:"string"}
  Date_to_Fetch = "2020/05/24" #@param {type:"date"}
  rv = get_CORS_SP3_Nav(Project_Root_Folder, Date_to_Fetch )


# References, Todo, Change Log & Bug Fixes

## PPK Related Tools:
* [RTKlib](https://colab.research.google.com/github/lidar532/ppkgeotag/blob/2020-1020-dev/RTKlib.ipynb)
* [CORS_lib](https://colab.research.google.com/github/lidar532/ppkgeotag/blob/2020-1020-dev/CORS_lib.ipynb)
* [PPK_Utilities](https://colab.research.google.com/github/lidar532/ppkgeotag/blob/2020-1020-dev/PPK_Utilities.ipynb)
* [PPK_Trajectory_Analysis](https://colab.research.google.com/github/lidar532/ppkgeotag/blob/2020-1020-dev/PPK_trajectory_analysis.ipynb)
* [PPK-2-PixPos](https://colab.research.google.com/github/lidar532/ppkgeotag/blob/2020-1020-dev/PPK_2_PixPos.ipynb)
* [Flash_Processing](https://colab.research.google.com/github/lidar532/flash-processing/blob/main/Flash_processing.ipynb)

## References
* [multiprocessing Basics](https://pymotw.com/2/multiprocessing/basics.html)
* [CORS Home](https://geodesy.noaa.gov/CORS/)
* [IGS (RINEX) File formats](https://kb.igs.org/hc/en-us/articles/201096516-IGS-Formats)
* [User Friendly CORS](https://www.ngs.noaa.gov/UFCORS/)
* [CORS Map](https://geodesy.noaa.gov/CORS_Map/)
* [Sortable list of CORS Stations](https://geodesy.noaa.gov/CORS/sort_sites.shtml)
* [ANTEX Antenna file ngs14.atx](https://www.ngs.noaa.gov/ANTCAL/LoadFile?file=ngs14.atx)
* [Juypter Widgets](https://ipywidgets.readthedocs.io/en/latest/index.html)
* [Python subprocess](https://docs.python.org/3/library/subprocess.html)

## Todo list
* Add code to gather and merge "same day" data. [(see teqc merge post here)](https://postal.unavco.org/pipermail/teqc/2014/001827.html)
* Switch to [Juypter Widgets](https://ipywidgets.readthedocs.io/en/latest/index.html)
* Add [multiprocessing](https://pymotw.com/2/multiprocessing/basics.html) to gather all the stations in parallel.

## Change Log & Bug Fixes
* 2020-1019
  * Added tool cells for gzip, gunzip, mv, RINEX time span extract
  * Added number of lines selector to the display file cell
  * Consoldated all the defs into one cell to make it easier to use stand-alone
  * Added tool to show the size of a Path_Name

* 2020-1018
  * Fixed problem where teqc wasn't loading and thus not allowing a RINEX file to be cut by time
* 2020-0622
  * Added input validity checking to time input strings.
  * Initial commit.